In [1]:
import PIL
from PIL import Image

In [2]:
cd D:\open_images_folder

D:\open_images_folder


In [3]:
import wget
url = 'https://raw.githubusercontent.com/openimages/dataset/master/downloader.py'
wget.download(url)

100% [################################################################################]     4K / 4K

'downloader.py'

In [4]:
!python downloader.py train_ImageID.txt --download_folder=D:\open_images_folder\Images_train --num_processes=5

In [5]:
!python downloader.py validation_ImageID.txt --download_folder=D:\open_images_folder\Images_validation --num_processes=5

In [6]:
!python downloader.py test_ImageID.txt --download_folder=D:\open_images_folder\Images_test --num_processes=5

In [7]:
def darknet_labels(ImageID, df):
    for an_ImageID in ImageID:
        image = PIL.Image.open(an_ImageID + '.jpg')
        width, height = image.size
        one_image = df.loc[df.ImageID == an_ImageID, ['ImageID', 'Label', 'XMin', 'XMax', 'YMin', "YMax"]]
        one_image['XMin_new'] = one_image.XMin.apply(lambda x: x*width)
        one_image['YMin_new'] = one_image.YMin.apply(lambda x: x*height)
        one_image['XMax_new'] = one_image.XMax.apply(lambda x: x*width)
        one_image['YMax_new'] = one_image.YMax.apply(lambda x: x*height)
    
        for index, row in one_image.iterrows():
            XMax_new =float(row['XMax_new'])
            XMin_new = float(row['XMin_new'])
            YMax_new =float(row['YMax_new'])
            YMin_new= float(row['YMin_new'])
            XMax_Norm = XMax_new - XMin_new
            YMax_Norm = YMax_new - YMin_new
            X_diff = int(XMax_Norm/2)
            Y_diff = int(YMax_Norm/2)
            XMin_Norm = XMin_new + X_diff
            YMin_Norm = YMin_new + Y_diff
            XMin_Norm = XMin_Norm/width
            YMin_Norm = YMin_Norm/height
            XMax_Norm = XMax_Norm/width
            YMax_Norm = YMax_Norm/height
            one_image.loc[index,'XMin_Norm'] = XMin_Norm
            one_image.loc[index,'YMin_Norm'] = YMin_Norm
            one_image.loc[index,'XMax_Norm'] = XMax_Norm
            one_image.loc[index,'YMax_Norm'] = YMax_Norm
        
        f = open(an_ImageID + ".txt","w+")
        for index, row in one_image.iterrows():
            each_row = str(row['Label']) + ' ' + str(row['XMin_Norm']) + ' ' + str(row['YMin_Norm']) + ' ' + str(row['XMax_Norm']) + ' ' + str(row['YMax_Norm']) +'\n'
            f.write(each_row)
        f.close()

In [8]:
import pandas as pd

df_train_final = pd.read_excel('train_annotation.xlsx')
df_train_final

,ImageID,Category,XMin,XMax,YMin,YMax
0,0001c3e14fe71e81,Man,0.262387,0.541667,0.227599,1.000000
1,0001c3e14fe71e81,Boy,0.028153,0.143018,0.252688,0.983871
2,0001c3e14fe71e81,Boy,0.145270,0.288288,0.333333,0.998208
3,0001c3e14fe71e81,Boy,0.183559,0.322072,0.458781,0.998208
4,0001c3e14fe71e81,Boy,0.235360,0.382883,0.370968,0.863799
...,...,...,...,...,...,...
93563,fff99eb8ac5cb649,Girl,0.559375,0.798750,0.256121,0.988701
93564,fff99eb8ac5cb649,Girl,0.609375,0.733125,0.500000,0.999058
93565,fff99eb8ac5cb649,Girl,0.733125,0.953750,0.363465,0.999058
93566,fff99eb8ac5cb649,Girl,0.743750,0.885625,0.277778,0.822976


In [9]:
df_train_final['Label'] = df_train_final.Category.map({'Man':'0', 'Woman':'1', 'Boy': '2', 'Girl':'3'})
df_train_final

,ImageID,Category,XMin,XMax,YMin,YMax,Label
0,0001c3e14fe71e81,Man,0.262387,0.541667,0.227599,1.000000,0
1,0001c3e14fe71e81,Boy,0.028153,0.143018,0.252688,0.983871,2
2,0001c3e14fe71e81,Boy,0.145270,0.288288,0.333333,0.998208,2
3,0001c3e14fe71e81,Boy,0.183559,0.322072,0.458781,0.998208,2
4,0001c3e14fe71e81,Boy,0.235360,0.382883,0.370968,0.863799,2
...,...,...,...,...,...,...,...
93563,fff99eb8ac5cb649,Girl,0.559375,0.798750,0.256121,0.988701,3
93564,fff99eb8ac5cb649,Girl,0.609375,0.733125,0.500000,0.999058,3
93565,fff99eb8ac5cb649,Girl,0.733125,0.953750,0.363465,0.999058,3
93566,fff99eb8ac5cb649,Girl,0.743750,0.885625,0.277778,0.822976,3


In [10]:
#Obtaining the 8000 ImageIDs for the 8000 train images
train_ImageID = list(set(list(df_train_final.ImageID)))
print(len(train_ImageID))
train_ImageID

8000


['21005d89f0bb66d8',
 '9c185ea6a764f81b',
 'fdaf440d091f70ca',
 '3e7e118c943ddeba',
 '1aceed18f1cbd68d',
 '4a518445d362dd6d',
 '1f4bb1c70eff620d',
 '29cb87dc80ddf385',
 '5d12b95be8295230',
 '82404c4f4213ba16',
 '180b68c72f03b388',
 '5eff72efbdf09cc1',
 'a1f08e8d8b3ede88',
 'e322920f35fd06ae',
 'ffb04b5c89afb4c2',
 'd43e955b75c9d378',
 '1f92e524dc328d5a',
 'b7596446df4dccc0',
 'c63025a0cdfa1474',
 'fa32c08fa9cde33a',
 '5b28d04cb96de178',
 '2509ae6e6c74f96f',
 '269fd47c637bfbb7',
 '443211fb6aaedf30',
 '98019b4198082f34',
 'cd6c9d096aff52ff',
 '86424fac6b2382b4',
 '67d01e4b33578373',
 '8fdcd10f90083260',
 'f41ced4937345b5e',
 '8ff0f158477434ee',
 '558d40bf4105a1ef',
 '50b160de0e77f70c',
 '5c0c6cf1d5c39a04',
 'ed261d8b94043595',
 'd2c9ddf2b3df75ca',
 '31eb68798c1f8518',
 '90097d56c66070d5',
 '3ffed6beac33cfb0',
 '6af8a8a742912554',
 'aaa6b209933a190d',
 '3226bc0ab72da1b5',
 '42b0df59ed040134',
 '471148c13a80f05c',
 '443c0a14eaf86970',
 '82f0fb7aebe1725d',
 '9a54a5566c1fb923',
 '081c3321731

In [11]:
cd D:\open_images_folder\Images_train

D:\open_images_folder\Images_train


In [12]:
darknet_labels(train_ImageID, df_train_final)

In [13]:
cd D:\open_images_folder

D:\open_images_folder


In [14]:
#Obtaining the 1000 ImageIDs for the 1000 validation images
df_validation_final = pd.read_excel('validation_annotation.xlsx')
df_validation_final['Label'] = df_validation_final.Category.map({'Man':'0', 'Woman':'1', 'Boy': '2', 'Girl':'3'})
validation_ImageID = list(set(list(df_validation_final.ImageID)))
print(len(validation_ImageID))

1000


In [15]:
cd D:\open_images_folder\Images_validation

D:\open_images_folder\Images_validation


In [16]:
darknet_labels(validation_ImageID, df_validation_final)

In [17]:
cd D:\open_images_folder

D:\open_images_folder


In [18]:
#Obtaining the 1000 ImageIDs for the 1000 test images
df_test_final = pd.read_excel('test_annotation.xlsx')
df_test_final['Label'] = df_test_final.Category.map({'Man':'0', 'Woman':'1', 'Boy': '2', 'Girl':'3'})
test_ImageID = list(set(list(df_test_final.ImageID)))
print(len(test_ImageID))

1000


In [19]:
cd D:\open_images_folder\Images_test

D:\open_images_folder\Images_test


In [20]:
darknet_labels(test_ImageID, df_test_final)

In [21]:
cd D:\open_images_folder

D:\open_images_folder
